# Notebook 02 – Data Preprocessing
## Cleaning, Feature Engineering, and Preparation for FinRL

This notebook preprocesses historical stock price data for:
- COMI
- AMOC
- SWDY

Steps:
1. Load raw stock data from Google Drive
2. Handle missing values
3. Compute daily returns
4. Add technical indicators
5. Save processed datasets for FinRL


In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required libraries
!pip install pandas numpy ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=12b6f48b655434cada4b63ba05444f3a5bcabbe774711edd543f680b08ffca48
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [3]:
import os
import pandas as pd
import numpy as np
import ta

BASE_DIR = '/content/drive/MyDrive/finrl-egx-multimodal'
RAW_DIR = os.path.join(BASE_DIR, 'data/stocks')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/stocks_processed')

os.makedirs(PROCESSED_DIR, exist_ok=True)

stocks = ['COMI', 'AMOC', 'SWDY']

In [4]:
def preprocess_stock(stock):
    file_path = os.path.join(RAW_DIR, f'{stock}.csv')
    df = pd.read_csv(file_path)

    # Convert Date
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)

    # Convert 'Adj Close' to numeric, coercing errors to NaN
    df['Adj Close'] = pd.to_numeric(df['Adj Close'], errors='coerce')

    # Handle missing values (after converting to numeric, NaNs from coerce will be handled)
    df.ffill(inplace=True) # Updated to use ffill() directly
    df.dropna(inplace=True)

    # Daily returns
    df['daily_return'] = df['Adj Close'].pct_change()

    # Technical indicators
    df['rsi'] = ta.momentum.RSIIndicator(df['Adj Close']).rsi()
    df['sma_20'] = ta.trend.SMAIndicator(df['Adj Close'], window=20).sma_indicator()
    df['sma_50'] = ta.trend.SMAIndicator(df['Adj Close'], window=50).sma_indicator()
    df['volatility'] = df['daily_return'].rolling(window=20).std()

    df.dropna(inplace=True)

    return df

In [5]:
# Run preprocessing and save results
for stock in stocks:
    print(f'Processing {stock}...')
    processed_df = preprocess_stock(stock)
    output_path = os.path.join(PROCESSED_DIR, f'{stock}_processed.csv')
    processed_df.to_csv(output_path, index=False)
    print(f'Saved: {output_path}')

Processing COMI...
Saved: /content/drive/MyDrive/finrl-egx-multimodal/data/stocks_processed/COMI_processed.csv
Processing AMOC...
Saved: /content/drive/MyDrive/finrl-egx-multimodal/data/stocks_processed/AMOC_processed.csv
Processing SWDY...
Saved: /content/drive/MyDrive/finrl-egx-multimodal/data/stocks_processed/SWDY_processed.csv


In [6]:
# Preview processed data
pd.read_csv(os.path.join(PROCESSED_DIR, 'COMI_processed.csv')).head()

,Date,Adj Close,Close,High,Low,Open,Volume,daily_return,rsi,sma_20,sma_50,volatility
0,2020-03-15,28.729263,31.47247314453125,34.14461898803711,31.447547912597656,34.937286376953125,17661326,-0.100841,15.340648,36.029904,37.541819,0.031654
1,2020-03-16,26.804270,29.363672256469727,31.25810432434082,28.341678619384766,31.47247314453125,5148437,-0.067005,12.860626,35.425101,37.321560,0.033519
2,2020-03-17,27.054565,29.637866973876953,30.659862518310547,28.665725708007812,29.363672256469727,4629706,0.009338,14.789543,34.820981,37.126694,0.033649
3,2020-03-18,25.866806,28.336692810058594,30.41558074951172,27.444316864013672,29.637866973876953,6239830,-0.043902,13.286506,34.172717,36.907254,0.034054
4,2020-03-19,27.773596,30.425552368164062,31.168367385864258,25.524959564208984,28.336692810058594,13080546,0.073716,26.245275,33.605231,36.707292,0.039590
